In [1]:
!pip install newspaper3k feedparser lxml[html_clean] -q

from google.colab import drive
drive.mount('/content/drive')

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 55.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 7.3 MB/s eta 0:00:00
Mounted at /content/drive


In [2]:
import requests
from bs4 import BeautifulSoup
from newspaper import Article
import pandas as pd
import time
import random
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.notebook import tqdm
import os

In [3]:
# --- CẤU HÌNH ---
TARGET_PER_GROUP = 1500
MAX_WORKERS = 5
OUTPUT_FOLDER = "/content/drive/My Drive/NLP Project/Data"
OUTPUT_FILE = os.path.join(OUTPUT_FOLDER, "vnexpress_full_dataset.csv")

category_groups = {
    "The_thao": [
        "https://vnexpress.net/the-thao",
        "https://vnexpress.net/bong-da",
        "https://vnexpress.net/tennis",
        "https://vnexpress.net/cac-mon-khac",
        "https://vnexpress.net/the-thao/hau-truong",
        "https://vnexpress.net/the-thao/marathon"
    ],
    "Suc_khoe": [
        "https://vnexpress.net/suc-khoe",
        "https://vnexpress.net/suc-khoe/cac-benh",
        "https://vnexpress.net/suc-khoe/tin-tuc",
        "https://vnexpress.net/suc-khoe/vaccine",
        "https://vnexpress.net/suc-khoe/khoe-dep",
        "https://vnexpress.net/suc-khoe/dinh-duong",
        "https://vnexpress.net/suc-khoe/ung-thu",
        "https://vnexpress.net/suc-khoe/dan-ong",
        "https://vnexpress.net/suc-khoe/phu-nu"
    ],
    "Giao_duc": [
        "https://vnexpress.net/giao-duc",
        "https://vnexpress.net/giao-duc/tuyen-sinh",
        "https://vnexpress.net/giao-duc/du-hoc",
        "https://vnexpress.net/giao-duc/hoc-tieng-anh",
        "https://vnexpress.net/giao-duc/giao-duc-40"
    ],
    "Phap_luat": [
        "https://vnexpress.net/phap-luat",
        "https://vnexpress.net/phap-luat/ho-so-pha-an",
        "https://vnexpress.net/phap-luat/tu-van"
    ],
    "Kinh_doanh": [
        "https://vnexpress.net/kinh-doanh",
        "https://vnexpress.net/kinh-doanh/quoc-te",
        "https://vnexpress.net/kinh-doanh/chung-khoan",
        "https://vnexpress.net/kinh-doanh/doanh-nghiep",
        "https://vnexpress.net/kinh-doanh/ebank",
        "https://vnexpress.net/kinh-doanh/vi-mo",
        "https://vnexpress.net/kinh-doanh/tien-cua-toi",
        "https://vnexpress.net/bat-dong-san"
    ],
    "Thu_gian": [
        "https://vnexpress.net/thu-gian/chuyen-la",
        "https://vnexpress.net/thu-gian/cuoi",
        "https://vnexpress.net/du-lich",
        "https://vnexpress.net/giai-tri/gioi-sao",
        "https://vnexpress.net/giai-tri/phim",
        "https://vnexpress.net/giai-tri/nhac"
    ],
    "Khoa_hoc_cong_nghe": [
        "https://vnexpress.net/khoa-hoc-cong-nghe",
        "https://vnexpress.net/khoa-hoc-cong-nghe/chuyen-doi-so",
        "https://vnexpress.net/khoa-hoc-cong-nghe/doi-moi-sang-tao",
        "https://vnexpress.net/khoa-hoc-cong-nghe/ai",
        "https://vnexpress.net/khoa-hoc-cong-nghe/vu-tru",
        "https://vnexpress.net/khoa-hoc-cong-nghe/the-gioi-tu-nhien",
        "https://vnexpress.net/khoa-hoc-cong-nghe/thiet-bi",
        "https://vnexpress.net/khoa-hoc-cong-nghe/cua-so-tri-thuc"
    ],
    "Xe": [
        "https://vnexpress.net/oto-xe-may",
        "https://vnexpress.net/oto-xe-may/thi-truong",
        "https://vnexpress.net/oto-xe-may/cam-lai",
        "https://vnexpress.net/oto-xe-may/v-car",
        "https://vnexpress.net/oto-xe-may/v-bike",
        "https://vnexpress.net/oto-xe-may/xe-dien",

    ],
    "Doi_song": [
        "https://vnexpress.net/doi-song",
        "https://vnexpress.net/doi-song/nhip-song",
        "https://vnexpress.net/doi-song/to-am",
        "https://vnexpress.net/doi-song/bai-hoc-song",
        "https://vnexpress.net/doi-song/cooking",
        "https://vnexpress.net/doi-song/tieu-dung"
    ],
    "The_gioi": [
        "https://vnexpress.net/the-gioi",
        "https://vnexpress.net/the-gioi/tu-lieu",
        "https://vnexpress.net/the-gioi/phan-tich",
        "https://vnexpress.net/the-gioi/nguoi-viet-5-chau",
        "https://vnexpress.net/the-gioi/cuoc-song-do-day",
        "https://vnexpress.net/the-gioi/quan-su"
    ]
}

In [4]:
USER_AGENTS = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Safari/605.1.15',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'
]

def get_random_header():
    return {'User-Agent': random.choice(USER_AGENTS)}

def get_links_from_page(category_url, page_num):
    try:
        url = f"{category_url}-p{page_num}"
        time.sleep(random.uniform(0.5, 1.5))
        response = requests.get(url, headers=get_random_header(), timeout=15)
        if response.status_code != 200: return []

        soup = BeautifulSoup(response.content, 'html.parser')
        links = set()
        for article in soup.select('h3.title-news a, h2.title-news a'):
            href = article.get('href')
            if href and '.html' in href and 'video' not in href and 'podcast' not in href:
                links.add(href)
        return list(links)
    except: return []

def crawl_content(url, label):
    try:
        config = Article(url).config
        config.browser_user_agent = random.choice(USER_AGENTS)
        config.request_timeout = 15
        article = Article(url, config=config)
        article.download()
        article.parse()

        if len(article.text) > 250: # Chỉ lấy bài dài > 250 ký tự
            return {
                "title": article.title,
                "content": article.text,
                "label": label,
                "url": url
            }
    except: return None
    return None

In [5]:
all_data = []

print(f" Bắt đầu Crawl (Target: {TARGET_PER_GROUP}/nhóm)...")
print(f" Dữ liệu sẽ lưu tại: {OUTPUT_FILE}")

for group_label, urls in category_groups.items():
    print(f"\n--- Đang xử lý nhóm: {group_label} ---")

    collected_links = set()
    for base_url in urls:
        if len(collected_links) >= TARGET_PER_GROUP: break

        print(f"  Quét: {base_url}")
        page = 1
        empty_count = 0

        while len(collected_links) < TARGET_PER_GROUP:
            new_links = get_links_from_page(base_url, page)

            if not new_links:
                empty_count += 1
                if empty_count > 3: break
            else:
                empty_count = 0
                collected_links.update(new_links)

            page += 1
            if page > 50: break

    final_links = list(collected_links)[:TARGET_PER_GROUP]
    print(f"    Đã chốt {len(final_links)} link. Bắt đầu tải nội dung...")

    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        futures = [executor.submit(crawl_content, link, group_label) for link in final_links]

        for future in tqdm(as_completed(futures), total=len(final_links), desc=f"DL {group_label}"):
            result = future.result()
            if result:
                all_data.append(result)
            time.sleep(0.05)

    df_temp = pd.DataFrame(all_data)
    df_temp.to_csv(OUTPUT_FILE, index=False, encoding="utf-8-sig")
    print(f" Đã lưu {len(df_temp)} dòng vào Drive.")

print(f"\n HOÀN TẤT TOÀN BỘ! File nằm trong Drive của bạn.")

 Bắt đầu Crawl (Target: 1500/nhóm)...
 Dữ liệu sẽ lưu tại: /content/drive/My Drive/NLP Project/Data/vnexpress_full_dataset.csv

--- Đang xử lý nhóm: The_thao ---
  Quét: https://vnexpress.net/the-thao
  Quét: https://vnexpress.net/bong-da
  Quét: https://vnexpress.net/tennis
  Quét: https://vnexpress.net/cac-mon-khac
  Quét: https://vnexpress.net/the-thao/hau-truong
  Quét: https://vnexpress.net/the-thao/marathon
    Đã chốt 1483 link. Bắt đầu tải nội dung...


DL The_thao:   0%|          | 0/1483 [00:00<?, ?it/s]

 Đã lưu 1476 dòng vào Drive.

--- Đang xử lý nhóm: Suc_khoe ---
  Quét: https://vnexpress.net/suc-khoe
  Quét: https://vnexpress.net/suc-khoe/cac-benh
  Quét: https://vnexpress.net/suc-khoe/tin-tuc
  Quét: https://vnexpress.net/suc-khoe/vaccine
  Quét: https://vnexpress.net/suc-khoe/khoe-dep
  Quét: https://vnexpress.net/suc-khoe/dinh-duong
  Quét: https://vnexpress.net/suc-khoe/ung-thu
  Quét: https://vnexpress.net/suc-khoe/dan-ong
    Đã chốt 1500 link. Bắt đầu tải nội dung...


DL Suc_khoe:   0%|          | 0/1500 [00:00<?, ?it/s]

 Đã lưu 2924 dòng vào Drive.

--- Đang xử lý nhóm: Giao_duc ---
  Quét: https://vnexpress.net/giao-duc
  Quét: https://vnexpress.net/giao-duc/tuyen-sinh
  Quét: https://vnexpress.net/giao-duc/du-hoc
    Đã chốt 1500 link. Bắt đầu tải nội dung...


...

 Đã lưu 10942 dòng vào Drive.

--- Đang xử lý nhóm: Doi_song ---
  Quét: https://vnexpress.net/doi-song
  Quét: https://vnexpress.net/doi-song/nhip-song
  Quét: https://vnexpress.net/doi-song/to-am
  Quét: https://vnexpress.net/doi-song/bai-hoc-song
    Đã chốt 1500 link. Bắt đầu tải nội dung...


DL Doi_song:   0%|          | 0/1500 [00:00<?, ?it/s]

 Đã lưu 12434 dòng vào Drive.

--- Đang xử lý nhóm: The_gioi ---
  Quét: https://vnexpress.net/the-gioi
  Quét: https://vnexpress.net/the-gioi/tu-lieu
  Quét: https://vnexpress.net/the-gioi/phan-tich
    Đã chốt 1500 link. Bắt đầu tải nội dung...


DL The_gioi:   0%|          | 0/1500 [00:00<?, ?it/s]

 Đã lưu 13928 dòng vào Drive.

 HOÀN TẤT TOÀN BỘ! File nằm trong Drive của bạn.
